In [342]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import time
import re
import random
from unidecode import unidecode
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [343]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [344]:
def GET_UA():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                ]
 
    return random.choice(uastrings)

In [345]:
def parse_url(url):
 
    headers = {'User-Agent': GET_UA()}
    content = None
 
    try:
        response = requests.get(url, headers=headers)
        ct = response.headers['Content-Type'].lower().strip()
 
        if 'text/html' in ct:
            content = response.content
            soup = BeautifulSoup(content)
        else:
            content = response.content
            soup = None
 
    except Exception as e:
        print('Error:', str(e))
 
    return content, soup, ct

In [346]:
players = pd.read_csv('playoff_players2023.txt')
player_id = players['Player-additional'].values.tolist()
player_name = players['Player'].values.tolist()
league_num =len(player_id)
i = 0
for name in player_name:
    player_name[i] = name.replace(" ", "_")
    i = i+1
print(player_name[0:10])
print(len(player_name))

['Devin_Booker', 'Anthony_Edwards', 'Jimmy_Butler', 'Trae_Young', 'Kevin_Durant', 'Jamal_Murray', 'Jayson_Tatum', 'Nikola_Jokić', 'Jaylen_Brown', 'Stephen_Curry']
212


In [347]:
players

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS▼,Player-additional
0,1,Devin Booker,SG,26,PHO,5,5,216,71,118,...,7,18,25,32,13,5,15,14,186,bookede01
1,2,Anthony Edwards,SG,21,MIN,5,5,199,55,114,...,3,22,25,26,9,10,8,13,158,edwaran01
2,3,Jimmy Butler,SF,33,MIA,4,4,140,54,86,...,8,14,22,20,7,1,7,6,146,butleji01
3,4,Trae Young,PG,24,ATL,5,5,190,51,121,...,5,14,19,51,9,2,19,8,145,youngtr01
4,5,Kevin Durant,SF,34,PHO,5,5,219,44,85,...,3,35,38,31,5,5,16,12,142,duranke01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,208,Lonnie Walker IV,SG,24,LAL,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,walkelo01
208,209,T.J. Warren,SF,29,PHO,2,0,6,0,1,...,0,0,0,0,0,0,0,2,0,warretj01
209,210,Peyton Watson,SG,20,DEN,1,0,4,0,1,...,1,0,1,1,0,0,0,0,0,watsope01
210,211,Ziaire Williams,SF,21,MEM,2,0,1,0,0,...,0,0,0,0,0,0,1,0,0,willizi02


In [348]:
base = 'https://www.basketball-reference.com/players/c/greendr01/gamelog-playoffs/'
# NBA season we will be analyzing
# URL page we will scraping (see image above)
url = base
print(url)
# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

https://www.basketball-reference.com/players/c/greendr01/gamelog-playoffs/


In [349]:
# use findALL() to get the column headers
soup.findAll('tr', limit=40)
# use getText()to extract the text we need into a list
i = 0
while i < 40:
    headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
    i = i +1
    if headers:
        idx = i
        i = 41

headers[0] = 'Player'
print(headers)

['Player', 'G', '2013 Playoffs', 'Series', 'Tm', '\xa0', 'Opp', 'G#', '\xa0', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']


In [350]:
# avoid the first header row
rows = soup.findAll('tr')
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
stats = pd.DataFrame(player_stats, columns = headers)
stats

ValueError: 31 columns passed, passed data had 30 columns

In [ ]:
stats = stats.dropna()
stats.reset_index(drop=True,inplace=True)
stats

In [ ]:
def get_stats(num):
    for person in range(num):
        # NBA season we will be analyzing
        player = player_id[person]
        person = person
        url = f'https://www.basketball-reference.com/players/c/{player}/gamelog-playoffs/'
        # URL page we will scraping
        print(url)
        time.sleep(2.95)
        # this is the HTML from the given URL
        content, soup, ct = parse_url(url)
        cols = [['G', 'Round', 'Series', 'Tm', ' ', 'Opp', 'G#', ' ', 'GS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']]
        # use findALL() to get the column headers
        soup.findAll('tr', limit=35)
        # use getText()to extract the text we need into a list
        i = 0
        try:
            while i < 40:
                headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
                i = i +1
                if headers:
                    idx = i
                    i = 41

            headers = headers[1:]

            rows = soup.findAll('tr')
            player_stats = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(len(rows))]
            stats = pd.DataFrame(player_stats, columns = cols)
            stats = stats.dropna()
            stats = stats.reset_index(drop=True)
            stats.insert(0, 'Player', player_name[person])
            stats.index = range(len(stats))
            if person == 0:
                data_core = stats
            else:
                data_core = pd.concat([data_core,stats])
            print(n,'/',person,player_name[person])
        except:
            print(person,'no playoff games')
    return data_core


In [ ]:
test = get_stats(len(player_name))

In [ ]:
test.dropna(inplace=True)
save = test.copy(deep=True)

In [ ]:
save

In [ ]:
test = test.reset_index(drop=True)
test

In [ ]:
headers

In [ ]:
test = test.reset_index(drop=True)
x = pd.DataFrame(test.values)
x.columns = headers
x.replace('', np.nan, inplace=True)
x = x.dropna(subset='Opp').reset_index(drop=True)
test = x

In [ ]:
test.columns = ['Player', 'G', 'Date', 'Series', 'Tm', 'H/A', 'Opp', 'G#', 'W/L',
       'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc',
       '+/-']

In [ ]:
test

In [ ]:
test['Date'] = pd.to_datetime(test['Date'])
data = test.copy(deep=True)
for i in range(len(data)):
    data.iloc[i,0] = unidecode(data.iloc[i,0]).replace('_',' ')
    
    

    #remove special characters

In [ ]:
data

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data['Player'][2]

In [ ]:
len(data['Player'].unique())

In [ ]:
info = pd.read_csv('player_index.csv')
for i in range(len(info)):
    info.iloc[i,1] = unidecode(info.iloc[i,1]).replace('_',' ')
print(data['Player'][2])

p2pos = {player:pos for player,pos in zip(info['Player'].to_list(),info['Pos'].to_list())}
pos = []
for i in range(len(data)):
    try:
        pos.append(p2pos[data.iloc[i][0]])
    except:
        pos.append('F')
pos

data['Pos'] = pos
data

In [ ]:
minutes =  data['MP'].to_list()

for i in range(len(minutes)):
    new = minutes[i].split(':')
    res = float(new[0])+(float(new[1])/60)
    minutes[i] = res
data['MP'] = minutes


In [ ]:


home = data['H/A'].values

for i in range(len(home)):
    if home[i] == '@':
        home[i] = 0
    else:
        home[i] = 1
home[0:10]


In [ ]:
data['H/A'] = home
data = data.replace('CHO','CHA')
data = data.replace('PHO','PHX')
data= data.replace('BRK','BKN')

WL = data['W/L'].values.astype(str)
margin = []
for i in range(len(WL)):
    input_string = WL[i]
    output = re.match(r"^[WL]\s*\(([+-]?\d+)\)", input_string).group(1)
    margin.append(output)

margin[0:10]
data['W/L']=margin
data['W/L'] = data['W/L'].astype(int)

In [ ]:
data['Date'].max()

In [ ]:
KM_vals = pd.read_csv('KM_vals.csv',index_col=0)

In [ ]:
KM_dict = dict(zip(KM_vals.Player,KM_vals.KM))

In [ ]:
data.Player.values[0][0]

In [ ]:
temp = []
for p in data.Player.values:
    if p[0] in KM_dict:
        temp.append(KM_dict[p[0]])
    else:
        temp.append(15)
data['KM'] = temp

In [ ]:
data[['FG%','3P%','FT%']] = data[['FG%','3P%','FT%']].replace(5.00,0)

In [ ]:
data.to_csv('pdata.csv',index=False)

In [ ]:
data

In [ ]:
data.loc[data.Player=='Stephen Curry'].sort_values(by='Date')